### Plot derived variables against original data 
To run this Notebook:
- Install Python and auv-python with instructions at https://bitbucket.org/MBARIMIke/auv-python
- `cd auv-python`
- `jupyter notebook`
- Select this file and execute the cells

In [ ]:
import os
import xarray as xr
import hvplot.pandas
import ipywidgets as widgets
import matplotlib.pyplot as plt
import pandas as pd
from logs2netcdfs import BASE_PATH, LOG_FILES

# Pick auv_name available from locally processed files
auv_picker = widgets.Dropdown(options=[f for f in sorted(os.listdir(BASE_PATH))
                                               if f != '.DS_Store'])
display(auv_picker)

In [ ]:
# Pick auv mission on local system
auv_name = auv_picker.value
missions_dir = os.path.join(BASE_PATH, auv_name, 'missionnetcdfs')
mission_picker = widgets.Dropdown(options=[f for f in sorted(os.listdir(missions_dir))
                                                   if f != '.DS_Store'])
display(mission_picker)

In [ ]:
mission = mission_picker.value

# Read all the netCDF file data into in-memory xarray objects saved in a dictionary
netcdfs_dir = os.path.join(BASE_PATH, auv_name, 'missionnetcdfs', mission)
ncs = {}
for lf in LOG_FILES:
    nc_file = lf.replace('.log', '.nc')
    try:
        ncs[nc_file] = xr.open_dataset(os.path.join(netcdfs_dir, nc_file))
    except FileNotFoundError:
        pass
    
cal_align = f"{auv_name}_{mission}.nc"
ncs[cal_align] = xr.open_dataset(os.path.join(netcdfs_dir, cal_align))
#display(ncs[cal_align])      # Uncomment to explore dataset

In [ ]:
%matplotlib notebook
pd.concat([ncs[cal_align]['filt_depth'].to_pandas().rename('filt_depth'), 
           ncs[cal_align]['nav_depth'].to_pandas().rename('nav_depth'),
           ncs['parosci.nc']['depth'].to_pandas().rename('depth')], axis=1).plot(style=['bs-','ro-','y^-'])

In [ ]:
# Print the variables in each data set
for nc_file, ds in ncs.items():
    print(f"{nc_file}: {[v for v in list(ds)]}")
    ##display(ds) # Uncomment to explore all the attributes

In [ ]:
# Pick the converted log to examine
ds_picker = widgets.Dropdown(options=ncs.keys())
display(ds_picker)

In [ ]:
# Pick the variable to plot
var_picker = widgets.Dropdown(options=list(ncs[ds_picker.value]))
display(var_picker)

In [ ]:
variable = var_picker.value

# Plot locally generated variable with the one created and served by SSDS
local_ds = ncs[ds_picker.value]


var = getattr(local_ds, variable)
try:
    d_var = getattr(ssds_ds, variable)
except AttributeError as e:
    d_var = xr.DataArray([])

df = pd.concat([local_ds[variable].to_pandas().rename(f"auv-python"),
                 ssds_ds[variable].to_pandas().rename(f"SSDS")], axis=1)
display(df.hvplot(ylabel=f"{var.attrs['long_name']} [{var.attrs.get('units', '')}]"))